In [57]:
import json
import pandas as pd
import spacy
import html.parser
import re
from flashgeotext.geotext import GeoText
geotext = GeoText(use_demo_data=True)
#nlpEn = spacy.load("en_core_web_lg") 
nlpFR = spacy.load('fr_core_news_md')


2020-09-10 18:05:16.146 | DEBUG    | flashgeotext.lookup:add:194 - cities added to pool
2020-09-10 18:05:16.151 | DEBUG    | flashgeotext.lookup:add:194 - countries added to pool
2020-09-10 18:05:16.152 | DEBUG    | flashgeotext.lookup:_add_demo_data:225 - demo data loaded for: ['cities', 'countries']


In [63]:
df = pd.read_json('data/dgOutput1000.json')
df.columns

Index(['bidDeadlineDate', 'buyerCountry', 'buyerName', 'contactAddress',
       'contactCity', 'contactCountry', 'contactEmail', 'contactFirstName',
       'contactLastName', 'contactPhone', 'contactState', 'country', 'cpvs',
       'creationDate', 'currency', 'description', 'dgmarketId',
       'estimatedCost', 'exclusion', 'exclusionWord', 'fileSource', 'lang',
       'noticeType', 'procurementId', 'procurementMethod', 'publicationDate',
       'sourceUrl', 'status', 'title', 'updateDate'],
      dtype='object')

In [59]:
def clean(Texte_Concat, accents = True):
    junk_space = [\
                    '<[^<]+?>', '\r\n', '!', ',', ';', \
                    '.', '?', '(', ')', '[', ']', '{', '}', '%', '*', '-', '_', '•', \
                    ':', '#', '€','–','>','<','«','»','+', '"', 'n°','N°', '●', '/', \
                    '►', '\n', '=', '“', '”', '…', '→', '‹', '≥', '⋅', '○',  \
                     '’', '‘', '—', '‑', 'Ø', 'Ä' \
                    ]
    Texte_without_junk = html.parser.unescape(Texte_Concat)
    Texte_without_junk = re.sub('<[^<]+?>', '', Texte_without_junk)

    for elmt in junk_space:
        Texte_without_junk = Texte_without_junk.replace(elmt,  ' ')
    return Texte_without_junk

In [60]:
df["text_clean"] = df["description"].apply(clean)

"Contract notice  Hosting of the is of the anact aract network     The service includes  hosting and monitoring of physical environments and production and pre production software  operations management  supervision  backups  restorations  archiving  data security  incident resolution  co  administration of certain elements of the technical infrastructure  supply of materials necessary for the performance of the contract  associated services  advice  technical support and engineering     This contract is divided into lots  no    Deadline for receipt of tenders or requests to participate  September 18  2020   5 00 pm    Language  s  official  s  of the EU  French    Avis No 20 100581     publié le 09 08 2020      Dépt    69 Anact departement finance gestion      Hébergement du SI du réseau Anact Aract       Avis de marché  Procédure Ouverte               Références complètes&nbsp        Avis  20 100581                Référence de TED        non disponible                 Annonce        

In [ ]:
#geoRes = geotext.extract(input_text= test, span_info=True)["cities"]
#geoRes
def traitement(x):
    cities = geotext.extract(input_text= x, span_info=True)["cities"]
    return(cities)
df['cities'] = df['description'].apply(traitement)

In [ ]:
df['cities'] 

In [ ]:
### que des données en français 
d["lang"].unique()

In [62]:
## entities  
text = test
doc = nlpFR(text)
for entity in doc.ents:
    print(entity.text, entity.label_)

Contract MISC
Hosting of the is of the anact aract network      MISC
The service includes  hosting and monitoring of physical environments and production and pre production software  operations management   MISC
This contract is divided into lots MISC
no    Deadline for receipt of tenders or requests to participate ORG
September 18 MISC
Language  s  official  s  of the EU MISC
French    Avis MISC
Dépt     MISC
Anact Aract PER
Avis de marché   MISC
TED         MISC
Annonce                        MISC
Département s   ORG
UELe MISC
Section I    PER
ADRESSES ORG
Département Finance ORG
Numéro national d'identification ORG
Mme Lamborot Delphine PER
Lyon LOC
F LOC
Adresse s   MISC
Adresse principale   http MISC
B8HOgrFBKt&amp v 1&amp selected 0 MISC
Les offres ou les demandes de participation MISC
B8HOgrFBKt&amp v 1&amp selected 0 MISC
ADJUDICATEUR ORG
Agence office national e   ORG
PRINCIPALE ORG
Services généraux des administrations publiques ORG
Section II PER
Objet                       